In [2]:
from __future__ import print_function
import os
from os import listdir
from os.path import join, isfile, splitext
import numpy as np
import pandas as pd
import cntk as C
from PIL import Image
import pickle
import time
import json
from cntk import load_model, combine
import cntk.io.transforms as xforms
from cntk.logging import graph
import getPatches
import cv2

RuntimeError: Unable to open ../models/shape_predictor_68_face_landmarks.dat

In [ ]:
def extract_features(image_path, loaded_model, output_nodes):   
    img = Image.open(image_path)       
    resized = img.resize((image_width, image_height), Image.ANTIALIAS)  
    
    bgr_image = np.asarray(resized, dtype=np.float32)[..., [2, 1, 0]]    
    hwc_format = np.ascontiguousarray(np.rollaxis(bgr_image, 2)) 
    
    arguments = {loaded_model.arguments[0]: [hwc_format]}    
    output = output_nodes.eval(arguments)   
    return output

In [4]:
pretrained_model_name = 'ResNet152_ImageNet_Caffe.model'
pretrained_model_path = '../models'
pretrained_node_name = 'pool5' 

label_mapping = {1: '1-Clear', 2: '2-Almost Clear', 3: '3-Mild', 4: '4-Moderate', 5: '5-Severe'}

img_path = '../data/labeled_test_images'
test_ground_truth = '../data/test_images_labels.csv' #If None, calculating RMSE on test set will be skipped
result_file = '../data/predicted_labels.csv'
patch_path = '../data/test_images_patches'
regression_model_path = '../models/cntk_regression.dat'
eye_cascade_model = '../models/haarcascade_eye.xml'

image_height = 224 # Here are the image height and width that the skin patches of the testing selfie are going to be resized to.
image_width  = 224 # They have to be the same as the ResNet-152 model requirement.
num_channels = 3

In [5]:
imageFiles = [f for f in listdir(img_path) if isfile(join(img_path, f))]
for imagefile in imageFiles:
    dim = getPatches.extract_patches(join(img_path, imagefile), {}, {}, patch_path) #extract_patches function is defined in getPatches.py

#load the stored regression model
read_model = pd.read_pickle(regression_model_path)
regression_model = read_model['model'][0]
train_regression = pickle.loads(regression_model)
#load the stored regression model
read_model = pd.read_pickle(regression_model_path)
regression_model = read_model['model'][0]
train_regression = pickle.loads(regression_model)
# get the score value for each patch

model_file  = os.path.join(pretrained_model_path, pretrained_model_name)
loaded_model  = load_model(model_file)
node_in_graph = loaded_model.find_by_name(pretrained_node_name)
output_nodes  = combine([node_in_graph.owner])

patch_score = dict()
for file in next(os.walk(patch_path))[2]:
    file_path = os.path.join(patch_path, file)
    # extract features from CNTK pretrained model
    score_features = extract_features (file_path, loaded_model, output_nodes)[0].flatten()
    # score the extracted features using trained regression model
    pred_score_label = train_regression.predict(score_features.reshape(1,-1))
    patch_score[file] = float("{0:.2f}".format(pred_score_label[0]))
# get the max score value among the patches and record the image name
image_patch_scores = {}
for key in patch_score:
    image_id = key.split("_landmark")[0]
    image_patch_scores_i = image_patch_scores.get(image_id, {"patch_name":[], "patch_score":[]})
    image_patch_scores_i["patch_name"].append(key)
    image_patch_scores_i["patch_score"].append(patch_score[key])
    image_patch_scores[image_id] = image_patch_scores_i

fp = open(result_file, 'w')
fp.write("Image_Name, Predicted_Label_Avg, Most_Severe_Patch\n")

for key in image_patch_scores:
    image_name = key.split("_landmark")[0]
    max_index = np.argmax(image_patch_scores[key]['patch_score'])
    Predicted_Label_Avg = np.mean(image_patch_scores[key]['patch_score'])
    Most_Severe_Patch = image_patch_scores[key]['patch_name'][max_index]
    fp.write('%s, %.4f, %s\n'%(image_name, Predicted_Label_Avg, Most_Severe_Patch))

fp.close()